In [22]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import model_utilities as mdl_util

import json
from datetime import datetime
import pandas as pd
import numpy as np

import pymongo
from pymongo import MongoClient

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [23]:
# Load in our parameters file
with open('../parameters.json') as f:
    params = json.load(f)

params

{'ftp_days': 50,
 'gtfs_period': 0,
 'database': 'muni_prediction_data',
 'avl_collection': 'avl_raw',
 'labeled_collection': 'labeled_trips',
 'chunk_collection': 'chunk_details',
 'duration_collection': 'trips_total_duration',
 'bus': '33',
 'direction': 0,
 'chunks': [2, 6],
 'chunk_2_collection': 'chunk_2_collection',
 'chunk_6_collection': 'chunk_6_collection'}

In [3]:
database = params['database']
duration_collection = params['duration_collection']

# Connect to our database/collection
client = MongoClient('localhost', 27017)
db = client[database]
duration_coll = db[duration_collection]

In [8]:
time_df = pd.DataFrame(list(duration_coll.find({},{'_id':0})))
trimed_df = time_df[time_df['duration'] < 4500]
trimed_df.head()

,duration,min_noon_sqr,min_since_midnight,start_timestamp,trip_id_iso
0,2341.0,481636,26,1.475912e+09,7253717_2016-10-08_4489K
1,2940.0,12321,609,1.475947e+09,7253845_2016-10-08_GHNYG
2,3450.0,729,747,1.475955e+09,7253837_2016-10-08_8Q6DP
3,3268.0,28900,890,1.475963e+09,7253830_2016-10-08_SR1KP
4,3274.0,96100,1030,1.475972e+09,7253823_2016-10-08_TDW9M


In [17]:
time_sort_df = trimed_df.sort_values('start_timestamp')
time_sort_df.reset_index(drop=True, inplace=True)

data_with_priors = []

for idx, row in time_sort_df.iterrows():

    if idx != 0:
        
        data_dict = {}
        
        prior = time_sort_df.iloc[idx-1]
        
        current_ts = row['start_timestamp']
        prior_ts = prior['start_timestamp']
        
        diff =  current_ts - prior_ts
        
        if diff < 1800:
            
            data_dict['min_noon_sqr'] = row['min_noon_sqr']
            data_dict['duration'] = row['duration']
            data_dict['prior_duration'] = prior['duration']
            
            data_with_priors.append(data_dict)
            
            

In [27]:
prior_df = pd.DataFrame(data_with_priors)
y = prior_df['duration'].values.reshape(-1,1)
X = prior_df[['min_noon_sqr', 'prior_duration']].values

In [21]:


scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
scaler_y = StandardScaler()
y_norm = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm)

# Build our model
ridge_cv = RidgeCV(alphas=[5,10,20,30], cv=20)
ridge_cv.fit(X_train, y_train)

#Score our model, R-Squared
r_squared = ridge_cv.score(X_test, y_test)

#Score our model, Root Mean Squared Error
y_predict = ridge_cv.predict(X_test)
rmse_norm = (mean_squared_error(y_test, y_predict))**.5
rmse_dev = scaler_y.inverse_transform([rmse_norm])
diff = rmse_dev[0] - scaler_y.mean_[0]

print ("R-squared: ", "{0:.2f}".format(r_squared))
print ('RMSE: ', "{0:.2f}".format(diff))

R-squared:  0.44
RMSE:  254.12


In [26]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

scaler_y = StandardScaler()
y_norm = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm)

regr = RandomForestRegressor(n_estimators=50, max_depth=2)

crossed = cross_validate(regr, X_train, y_train.flatten(), cv=20, 
                         scoring=['neg_mean_squared_error','r2'], return_train_score=False)

r_squared = crossed['test_neg_mean_squared_error'].mean()

#Score our model, Root Mean Squared Error
rmse_norm = (abs(crossed['test_neg_mean_squared_error'].mean()))**.5
rmse_dev = scaler_y.inverse_transform([rmse_norm])
diff = rmse_dev[0] - scaler_y.mean_[0]

print ("R-squared: ", "{0:.2f}".format(r_squared))
print ('RMSE: ', "{0:.2f}".format(diff))

ValueError: Found input variables with inconsistent numbers of samples: [1346, 1907]